In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from numpy import errstate,isneginf,array
import datetime
import os
import yaml

import matplotlib.pyplot as plt
import cmocean as cm    

from xgboost import XGBRegressor

In [2]:
#This file contains configuration details like API keys and passwords
global_vars = yaml.safe_load(open('../config.yml', 'r') )

In [3]:
#This has custom functions - log transform
%run ./00_custom_functions.ipynb

In [4]:
#Set base folders
result_folder = global_vars['reconstruction_folder']
data_folder_root = global_vars['download_folder']
print(result_folder)

#This variable sets the output file type. 
#When using cloud storage, it is recommended to use ARCO (Analysis-Ready Cloud-Optimized) formats like Zarr over NetCDF
output_file_type = '.zarr' if data_folder_root[0:5] == 'gs://' else '.nc'
print(f'Files will be outputed as: {output_file_type}')

gs://leap-persistent/galenmckinley/reconstructions
Files will be outputed as: .zarr


In [5]:
## PATHS FOR PROCESSED DATA
## must update these file names when extending training

#Set location of input files (path from root above)
#Note that these were processed such that they already share a coordinate set
sst_processed  = data_folder_root + 'SST/processed/SST_NOAA_OI-V2-1x1_198201-202304.nc'
sss_processed  = data_folder_root + 'SSS/processed/SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_198201-202304.nc' #previously 202303
mld_processed  = data_folder_root + 'MLD/processed/MLD_IFREMER-deBoyer_DT02-c1m-1x1_198201-202304.nc'
chl_processed  = data_folder_root + 'CHL/processed/CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_198201-202304.nc'
pco2_processed = data_folder_root + 'pCO2/processed/pCO2_LEAP_SOCAT-ERA5-fco2-weighted-gridded_198201-202212.nc' #updated for fco2
xco2_processed = data_folder_root + 'xCO2/processed/xCO2_NOAA_xCO2-mm-gl-monthly_198201-202304.nc'
#add additional sources if desired
sst_processed_option2 = data_folder_root + 'SST/processed/SST_ECMWF_ERA5-monthly-reanalysis-1x1-SST_198201-202304.nc'
sst_processed_option3 = data_folder_root + 'SST/processed/SST_JMA_JRA55-do-monthly-reanalysis-SST_198201-202304.nc'
mld_processed_option2 = data_folder_root + 'MLD/processed/MLD_UCSD-Argo_MLD-dt-mean-1x1_198201-202304.nc'
list_for_df = [sst_processed, sss_processed, mld_processed, chl_processed, pco2_processed, xco2_processed
              ,sst_processed_option2, sst_processed_option3, mld_processed_option2
              ] 

In [6]:
#This is where we set parameters for the ML algorithm for finding the long term pco2 mean feature

#The next variable is for the XGBoost method for both pCO2 Residual and creating the long term pCo2 mean feature. They were determined via a grid search in previous iterations. 
best_params = {'max_depth': 9, 'n_estimators': 1000} 
random_seed = 47  #Set the random seeds used for training
jobs = -1         #Number of cores you have access to for model training; -1 for all available ones

#This variable is a list of features used for the Long Term pCO2 mean machine learning
feature_sel = ['sst','sst_anomaly','sss','sss_anomaly','chl_log','chl_log_anomaly','mld_log','xco2_trend','A','B','C','T0','T1']
target_sel = ['fco2']  #previously was pco2
sst_variable_option = ['sst']  #the name of the SST variable to use for calculating the residual component of the target

# Create Features
### Base Features

In [7]:
xrfull = xr.merge([xr_open_dataset_custom(f) for f in list_for_df], compat='broadcast_equals')
xrfull.attrs = "" #just removing attribute details since wont be accurate anymore
#xrfull

Encountered an error - trying with gs://leap-persistent/galenmckinley/online_data/SST/processed/SST_NOAA_OI-V2-1x1_198201-202304.zarr...
Success.
Encountered an error - trying with gs://leap-persistent/galenmckinley/online_data/SSS/processed/SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_198201-202304.zarr...
Success.
Encountered an error - trying with gs://leap-persistent/galenmckinley/online_data/MLD/processed/MLD_IFREMER-deBoyer_DT02-c1m-1x1_198201-202304.zarr...
Success.
Encountered an error - trying with gs://leap-persistent/galenmckinley/online_data/CHL/processed/CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_198201-202304.zarr...
Success.
Encountered an error - trying with gs://leap-persistent/galenmckinley/online_data/pCO2/processed/pCO2_LEAP_SOCAT-ERA5-fco2-weighted-gridded_198201-202212.zarr...
Success.
Encountered an error - trying with gs://leap-persistent/galenmckinley/online_data/xCO2/processed/xCO2_NOAA_xCO2-mm-gl-monthly_198201-202304.zarr...
Success.
Encountered 

In [8]:
#next add derived (logs). 
#Divide by zero warnings are OK due the how the numpy log10 function is used and handled in the function.
xrfull = xrfull.assign( mld_log = log_or_0_xr(xrfull.mld, 'mld_log') 
                       ,chl_log = log_or_0_xr(xrfull.chl, 'chl_log')
                       ,mld_argo_log = log_or_0_xr(xrfull.mld_argo, 'mld_argo_log')
                      )

/srv/conda/envs/notebook/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log10
  return func(*(_execute_task(a, cache) for a in args))
/srv/conda/envs/notebook/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log10
  return func(*(_execute_task(a, cache) for a in args))


In [9]:
%%time
#add anomalies fields
anomalies = xrfull.chunk(-1).groupby("time.month") - xrfull.chunk(-1).groupby("time.month").mean("time") #using chunk(-1) to avoid errors with different chunk sizes in the groups
anomalies = anomalies.get(['sst','sss','chl_log', 'sst_era5', 'sst_jra55']) #just need SST, SSS, CHL anomalies
anomalies = anomalies.drop('month') 
anomalies = anomalies.rename({ 'sst': 'sst_anomaly'
                              ,'sss':'sss_anomaly'
                              ,'chl_log':'chl_log_anomaly'
                              ,'sst_era5':'sst_era5_anomaly'
                              ,'sst_jra55':'sst_jra55_anomaly'
                             }) 
#anomalies
xrfull = xrfull.merge(anomalies, compat='identical')

CPU times: user 308 ms, sys: 902 µs, total: 309 ms
Wall time: 309 ms


<timed exec>:4: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.


In [10]:
#add time and space derivations
xrfull = xrfull.assign( days_idx = xrfull.time.dt.dayofyear 
                       ,lon_rad = np.radians(xrfull.xlon)
                       ,lat_rad = np.radians(xrfull.ylat)
                      )
xrfull = xrfull.assign( T0 = np.cos(xrfull.days_idx * 2 * np.pi / 365)
                       ,T1 = np.sin(xrfull.days_idx * 2 * np.pi / 365)
                       ,A  = np.sin(xrfull.lat_rad)
                       ,B  = np.cos(xrfull.lat_rad)*np.sin(xrfull.lon_rad)
                       ,C  = -np.cos(xrfull.lat_rad)*np.cos(xrfull.lon_rad)
                      )

### Set Data Fields/Points For Learning Long Term Mean

In [11]:
%%time
xr_for_ocean_co2_mean = xrfull.get(feature_sel + target_sel)  #Note - we train on all available data (no time slice is used here)
df_for_ocean_co2_mean = xr_for_ocean_co2_mean.to_dataframe() #expensive function; need lots of RAM 
df_for_ocean_co2_mean_to_train = df_for_ocean_co2_mean[(~df_for_ocean_co2_mean.isna().any(axis=1))]  #only keep points that are not null 
print(f'Number of points in time/space for training: {df_for_ocean_co2_mean_to_train.shape[0]}')
#df_for_ocean_co2_mean_to_train

df_for_ocean_co2_mean_to_predict_temp = df_for_ocean_co2_mean.loc[:,feature_sel]
df_for_ocean_co2_mean_to_predict = df_for_ocean_co2_mean_to_predict_temp[(~df_for_ocean_co2_mean_to_predict_temp.isna().any(axis=1))]  #only predict on points with all variables globally
print(f'Number of points in time/space available to reconstruct: {df_for_ocean_co2_mean_to_predict.shape[0]}')
#df_for_ocean_co2_mean_to_predict

Number of points in time/space for training: 312201
Number of points in time/space available to reconstruct: 18346460
CPU times: user 8.88 s, sys: 3.15 s, total: 12 s
Wall time: 13.9 s


### ML for Ocean CO2_Long_Term_Mean Feature

In [12]:
%%time
X_train = df_for_ocean_co2_mean_to_train.loc[:,feature_sel]
y_train = df_for_ocean_co2_mean_to_train.loc[:,target_sel]

model = XGBRegressor(random_state=random_seed, **best_params, n_jobs=jobs)
print(f'Training started on '+datetime.datetime.now().strftime('%Y-%m-%d %H:%M')+'...')
model.fit(X_train, y_train)    #training on all data with no cross validation because we are only calculating a long term average
                               #Model evalation for pco2 residual is in the next script
print('Training completed on '+datetime.datetime.now().strftime('%Y-%m-%d %H:%M')+'. Predicting...')
ocean_co2_for_mean_recon = model.predict(df_for_ocean_co2_mean_to_predict)
print("Complete")  #Training and predicting may take 15 minutes

Training started on 2024-02-17 17:39...
Training completed on 2024-02-17 17:39. Predicting...
Complete
CPU times: user 11min 49s, sys: 688 ms, total: 11min 50s
Wall time: 5min 59s


In [13]:
#average across time and add back to dataset 
ocean_co2_for_mean_recon_xr = pd.DataFrame(ocean_co2_for_mean_recon,index=df_for_ocean_co2_mean_to_predict.index,columns=['ocean_co2_recon_for_mean']).to_xarray()
xrfull = xrfull.merge(ocean_co2_for_mean_recon_xr, compat='identical') #add back to full set
xrfull = xrfull.assign(ocean_co2_mean = xrfull.ocean_co2_recon_for_mean.mean('time'))

In [14]:
#Export if desired. Some extra code to find the dates used
if True:
    min_yearmonth = str(ocean_co2_for_mean_recon_xr.time.min().data.astype('datetime64[s]').item().strftime('%Y%m')) #just gets the min date from the xarray in YYYYMM format
    max_yearmonth = str(ocean_co2_for_mean_recon_xr.time.max().data.astype('datetime64[s]').item().strftime('%Y%m')) 
    ocean_co2_long_term_mean = ocean_co2_for_mean_recon_xr.mean('time')
    ocean_co2_long_term_mean = ocean_co2_long_term_mean.rename({'ocean_co2_recon_for_mean': str(target_sel[0])+'_mean'})
    ocean_co2_output_name = 'pCO2_LEAP_XGBoost-'+str(target_sel[0])+'-long-term-mean-from-'+min_yearmonth+'-to-'+max_yearmonth+'.nc'
    output_xarray_with_date(ocean_co2_long_term_mean, result_folder, ocean_co2_output_name, with_date=False,  filetype=output_file_type)

Saved pCO2_LEAP_XGBoost-fco2-long-term-mean-from-198201-to-202304.zarr to gs://leap-persistent/galenmckinley/reconstructions


### Residual (pCO2 T and NonT Features)

In [15]:
xrfull = xrfull.assign(ocean_co2_T = xrfull['ocean_co2_mean'] * np.exp(0.0413 * (xrfull[sst_variable_option[0]] - xrfull[sst_variable_option[0]].mean("time"))) ) # Wanninkhof et al. 2022
    #previously: xrfull = xrfull.assign(ocean_co2_T = xrfull['ocean_co2_mean'] * np.exp(0.0423* (xrfull.sst - xrfull.sst.mean("time"))) ) # Takahashia et al, 2004
xrfull = xrfull.assign(ocean_co2_nonT = xrfull[target_sel[0]] - xrfull.ocean_co2_T)

In [16]:
#Perform some clean up before exporting

xrfull = xrfull.drop_vars(['ocean_co2_recon_for_mean'])  #optionally could keep/drop this variable
xrfull = xrfull.rename({'ocean_co2_T': str(target_sel[0])+'_T'
                        ,'ocean_co2_nonT': str(target_sel[0])+'_nonT'
                        ,'ocean_co2_mean': str(target_sel[0])+'_mean'
                        #,'ocean_co2_recon_for_mean': str(target_sel[0])+'_recon_for_mean'
                       })

#add attributes as needed here
xrfull.attrs['description'] = "fCO2-Residual dataset for machine learning"
xrfull.attrs['formula_for_co2_residual'] = 'uses 0.0413 factor from Wanninkhof et al. 2022 and temperature variable: '+ str(sst_variable_option[0])
xrfull.attrs['training_features_for_co2_mean'] = str(feature_sel)
xrfull.attrs['hyperparameters_for_co2_mean'] = str(best_params)
xrfull.attrs['created'] = str(datetime.datetime.now())
#xrfull.attrs['methodology'] = "Bennington et al. (2022), JAMES"

In [17]:
xrfull

<xarray.Dataset>
Dimensions:            (time: 496, xlon: 360, ylat: 180)
Coordinates:
  * time               (time) datetime64[ns] 1982-01-15 ... 2023-04-15
  * xlon               (xlon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * ylat               (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Data variables: (12/29)
    sst                (time, ylat, xlon) float32 dask.array<chunksize=(496, 180, 360), meta=np.ndarray>
    sss                (time, ylat, xlon) float32 dask.array<chunksize=(496, 173, 360), meta=np.ndarray>
    mld                (time, ylat, xlon) float64 dask.array<chunksize=(62, 45, 90), meta=np.ndarray>
    chl                (time, ylat, xlon) float64 dask.array<chunksize=(496, 180, 360), meta=np.ndarray>
    fco2               (time, ylat, xlon) float32 dask.array<chunksize=(496, 180, 360), meta=np.ndarray>
    pco2               (time, ylat, xlon) float64 dask.array<chunksize=(496, 180, 360), meta=np.ndarray>
    ...                 ...
    A                  (ylat) float64 -1.0 -0.9997 -0.999 ... 0.999 0.9997 1.0
    B                  (ylat, xlon) float64 -7.615e-05 -0.0002284 ... 7.615e-05
    C                  (ylat, xlon) float64 0.008726 0.008724 ... 0.008726
    fco2_mean          (ylat, xlon) float32 nan nan nan nan ... nan nan nan nan
    fco2_T             (ylat, xlon, time) float32 dask.array<chunksize=(180, 360, 496), meta=np.ndarray>
    fco2_nonT          (time, ylat, xlon) float32 dask.array<chunksize=(496, 180, 360), meta=np.ndarray>
Attributes:
    description:                     fCO2-Residual dataset for machine learning
    formula_for_co2_residual:        uses 0.0413 factor from Wanninkhof et al...
    training_features_for_co2_mean:  ['sst', 'sst_anomaly', 'sss', 'sss_anoma...
    hyperparameters_for_co2_mean:    {'max_depth': 9, 'n_estimators': 1000}
    created:                         2024-02-17 17:45:11.799037

In [18]:
#export out (may be 3GB unless some variables dropped)
output_xarray_with_date(xrfull, result_folder+'', 'pCO2_LEAP_'+str(target_sel[0])+'-residual-full-dataset-preML', filetype=output_file_type)

Saved pCO2_LEAP_fco2-residual-full-dataset-preML_198201-202304.zarr to gs://leap-persistent/galenmckinley/reconstructions
